<a href="https://colab.research.google.com/github/Krishnaa548/Mlops/blob/main/Neural_Network_SMS_Text_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd # Import pandas library
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
train_df = pd.read_csv('/content/spam.csv', encoding='latin1')  # Specify 'latin1' encoding
test_df = pd.read_csv('/content/spam.csv', encoding='latin1')   # Specify 'latin1' encoding

In [ ]:
!pip install chardet
import chardet

with open('/content/spam.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))

print(result) # This will show the detected encoding

train_df = pd.read_csv('/content/spam.csv', encoding=result['encoding'])
test_df = pd.read_csv('/content/spam.csv', encoding=result['encoding'])

{'encoding': 'Windows-1252', 'confidence': 0.7272080023536335, 'language': ''}


In [ ]:
print(train_df.columns) # Print the current column names.

Index(['label', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')


In [ ]:
# Map labels to binary values: ham -> 0, spam -> 1.
label_map = {'ham': 0, 'spam': 1}
# Access the correct column name instead of 'label'

# Assuming your column is named 'v1' in the original CSV
# If it's different, replace 'v1' with the actual column name
train_df['Unnamed: 2'] = train_df['Unnamed: 2'].map(label_map)
test_df['Unnamed: 2'] = test_df['Unnamed: 2'].map(label_map)

# Rename column 'v1' to 'label' and 'v2' to 'message' for clarity.
train_df.rename(columns={'Unnamed: 2': 'label', 'v2': 'message'}, inplace=True)
test_df.rename(columns={'Unnamed: 2': 'label', 'v2': 'message'}, inplace=True)

In [ ]:
# Define the vocabulary size, maximum sequence length, and embedding dimension
vocab_size = 10000  # Example value, adjust as needed
max_length = 100    # Example value, adjust as needed
embedding_dim = 16   # Example value, adjust as needed

# Initialize and fit the tokenizer on the training messages.
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(train_df['message'])

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping
import chardet

# Detect encoding
with open('/content/spam.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))

# Read the CSV file with the detected encoding
df = pd.read_csv('/content/spam.csv', encoding=result['encoding'])

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42) # 80% train, 20% test

# Check and map labels if necessary
if train_df['v1'].dtypes != np.int64:
    label_map = {'ham': 0, 'spam': 1}
    train_df['v1'] = train_df['v1'].map(label_map)
    test_df['v1'] = test_df['v1'].map(label_map)

# Rename columns
train_df.rename(columns={'v1': 'label', 'v2': 'message'}, inplace=True)
test_df.rename(columns={'v1': 'label', 'v2': 'message'}, inplace=True)

# ... (Rest of your code for tokenization, model building, and training remains the same) ...

In [ ]:
# Convert text messages to sequences of integers.
train_sequences = tokenizer.texts_to_sequences(train_df['message'])
test_sequences = tokenizer.texts_to_sequences(test_df['message'])

In [ ]:
# Pad the sequences so they are all the same length.
X_train = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')


In [ ]:

# Get labels as NumPy arrays.
y_train = np.array(train_df['label'])
y_test = np.array(test_df['label'])

In [ ]:
# Create a simple sequential model.
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    GlobalAveragePooling1D(),
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer: sigmoid activation gives probability of spam.
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Compile the model using binary crossentropy since we have a binary classification.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ ?                           │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Use early stopping to avoid overfitting.
early_stop = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:

# Train the model.
history = model.fit(X_train, y_train,
                    epochs=20,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stop],
                    verbose=2)

Epoch 1/20
140/140 - 2s - 17ms/step - accuracy: 0.8661 - loss: 0.3832 - val_accuracy: 0.8655 - val_loss: 0.3678
Epoch 2/20
140/140 - 1s - 4ms/step - accuracy: 0.8661 - loss: 0.3605 - val_accuracy: 0.8655 - val_loss: 0.3597
Epoch 3/20
140/140 - 1s - 4ms/step - accuracy: 0.8661 - loss: 0.3497 - val_accuracy: 0.8655 - val_loss: 0.3460
Epoch 4/20
140/140 - 1s - 7ms/step - accuracy: 0.8661 - loss: 0.3270 - val_accuracy: 0.8655 - val_loss: 0.3160
Epoch 5/20
140/140 - 1s - 9ms/step - accuracy: 0.8759 - loss: 0.2586 - val_accuracy: 0.8915 - val_loss: 0.2082
Epoch 6/20
140/140 - 1s - 6ms/step - accuracy: 0.9542 - loss: 0.1355 - val_accuracy: 0.9722 - val_loss: 0.1181
Epoch 7/20
140/140 - 1s - 4ms/step - accuracy: 0.9782 - loss: 0.0772 - val_accuracy: 0.9713 - val_loss: 0.0934
Epoch 8/20
140/140 - 1s - 4ms/step - accuracy: 0.9827 - loss: 0.0594 - val_accuracy: 0.9794 - val_loss: 0.0854
Epoch 9/20
140/140 - 1s - 4ms/step - accuracy: 0.9868 - loss: 0.0473 - val_accuracy: 0.9776 - val_loss: 0.0744


In [ ]:

def predict_message(message):
    """
    Predicts whether a given SMS message is ham or spam.

    Args:
        message (str): The SMS message to classify.

    Returns:
        list: A two-element list where the first element is the probability (0 to 1)
              that the message is spam, and the second element is the string 'spam'
              if the message is more likely spam or 'ham' otherwise.
    """

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping
import chardet
from sklearn.model_selection import train_test_split

# Detect encoding
with open('/content/spam.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))

# Read the CSV file with the detected encoding
df = pd.read_csv('/content/spam.csv', encoding=result['encoding'])

# Split data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42) # 80% train, 20% test

# Check and map labels if necessary
if train_df['v1'].dtypes != np.int64:
    label_map = {'ham': 0, 'spam': 1}
    train_df['v1'] = train_df['v1'].map(label_map)
    test_df['v1'] = test_df['v1'].map(label_map)

# Rename columns
train_df.rename(columns={'v1': 'label', 'v2': 'message'}, inplace=True)
test_df.rename(columns={'v1': 'label', 'v2': 'message'}, inplace=True)

# Define the vocabulary size, maximum sequence length, and embedding dimension
vocab_size = 10000  # Example value, adjust as needed
max_length = 100    # Example value, adjust as needed
embedding_dim = 16   # Example value, adjust as needed

# Initialize and fit the tokenizer on the training messages.
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(train_df['message'])

# Convert text messages to sequences of integers.
train_sequences = tokenizer.texts_to_sequences(train_df['message'])
test_sequences = tokenizer.texts_to_sequences(test_df['message'])

# Pad the sequences so they are all the same length.
X_train = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

# Get labels as NumPy arrays.
y_train = np.array(train_df['label'])
y_test = np.array(test_df['label'])

# Create a simple sequential model.
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    GlobalAveragePooling1D(),
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer: sigmoid activation gives probability of spam.
])

# Compile the model using binary crossentropy since we have a binary classification.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Use early stopping to avoid overfitting.
early_stop = EarlyStopping(monitor='val_loss', patience=3)

# Train the model.
history = model.fit(X_train, y_train,
                    epochs=20,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stop],
                    verbose=2)

def predict_message(message):
    """
    Predicts whether a given SMS message is ham or spam.

    Args:
        message (str): The SMS message to classify.

    Returns:
        list: A two-element list where the first element is the probability (0 to 1)
              that the message is spam, and the second element is the string 'spam'
              if the message is more likely spam or 'ham' otherwise.
    """
    sequence = tokenizer.texts_to_sequences([message])
    padded = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')

    # Get prediction probability (the output is probability of spam since spam is encoded as 1).
    prob = model.predict(padded)[0][0]

    # Determine the label based on a 0.5 threshold.
    label = 'spam' if prob >= 0.5 else 'ham'
    return [float(prob), label]



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_1           │ ?                           │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
140/140 - 3s - 20ms/step - accuracy: 0.8607 - loss: 0.3884 - val_accuracy: 0.8655 - val_loss: 0.3674
Epoch 2/20
140/140 - 1s - 4ms/step - accuracy: 0.8661 - loss: 0.3603 - val_accuracy: 0.8655 - val_loss: 0.3622
Epoch 3/20
140/140 - 1s - 4ms/step - accuracy: 0.8661 - loss: 0.3523 - val_accuracy: 0.8655 - val_loss: 0.3500
Epoch 4/20
140/140 - 1s - 5ms/step - accuracy: 0.8661 - loss: 0.3375 - val_accuracy: 0.8655 - val_loss: 0.3312
Epoch 5/20
140/140 - 1s - 5ms/step - accuracy: 0.8661 - loss: 0.2958 - val_accuracy: 0.8655 - val_loss: 0.2764
Epoch 6/20
140/140 - 1s - 4ms/step - accuracy: 0.8986 - loss: 0.2115 - val_accuracy: 0.9318 - val_loss: 0.1744
Epoch 7/20
140/140 - 1s - 5ms/step - accuracy: 0.9641 - loss: 0.1163 - val_accuracy: 0.9749 - val_loss: 0.1121
Epoch 8/20
140/140 - 1s - 5ms/step - accuracy: 0.9803 - loss: 0.0727 - val_accuracy: 0.9758 - val_loss: 0.0872
Epoch 9/20
140/140 - 1s - 4ms/step - accuracy: 0.9838 - loss: 0.0548 - val_accuracy: 0.9776 - val_loss: 0.0782


In [ ]:

# --- Example Tests ---
# You can test the function with sample messages.

print(predict_message("Congratulations! You've won a free ticket to Bahamas. Call now!"))
print(predict_message("Hey, are we still on for lunch today?"))

None
None


In [ ]:
# --- Example Tests ---
# You can test the function with sample messages.

print(predict_message("Congratulations! You've won a free ticket to Bahamas. Call now!"))
print(predict_message("Hey, are we still on for lunch today?"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[0.9308504462242126, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[0.0024807313457131386, 'ham']
